In [26]:
import time
import asyncio

In [11]:
def fun1(x):
    print(x**2)
    time.sleep(3)
    print('fun1 завершена')


def fun2(x):
    print(x**0.5)
    time.sleep(3)
    print('fun2 завершена')


def main():
    fun1(4)
    fun2(4)

start = time.time()

main()

end = time.time()
print(f"Time: {end - start}")

16
fun1 завершена
2.0
fun2 завершена
Time:6.006633758544922


In [32]:
async def fun1(x):
    print(x**2)
    await asyncio.sleep(3)
    print('fun1 завершена')


async def fun2(x):
    print(x**0.5)
    await asyncio.sleep(3)
    print('fun2 завершена')


async def main():
    task1 = asyncio.create_task(fun1(4))
    task2 = asyncio.create_task(fun2(4))

    await task1
    await task2



start = time.time()

await main()

end = time.time()
print(f"Time: {end - start}")

16
2.0
fun1 завершена
fun2 завершена
Time:3.0155370235443115


#  "Во многой мудрости много печали; и кто умножает познания, умножает скорбь" (Еккл. 1:18)

In [36]:
#Попробуем прокинуть не задачи, а корутины
async def fun1(x):
    print(x**2)
    await asyncio.sleep(3)
    print('fun1 завершена')


async def fun2(x):
    print(x**0.5)
    await asyncio.sleep(3)
    print('fun2 завершена')


async def main():
    await fun1(4)
    await fun2(4)


start = time.time()

await main()

end = time.time()
print(f"Time: {end - start}")

16
fun1 завершена
2.0
fun2 завершена
Time: 6.024572849273682


In [34]:
async def fun1(x):
    print(x**2)
    time.sleep(3)
    print('fun1 завершена')


async def fun2(x):
    print(x**0.5)
    time.sleep(3)
    print('fun2 завершена')


async def main():
    task1 = asyncio.create_task(fun1(4))
    task2 = asyncio.create_task(fun2(4))

    await task1
    await task2



start = time.time()

await main()

end = time.time()
print(f"Time: {end - start}")

16
fun1 завершена
2.0
fun2 завершена
Time: 6.003616571426392


D:\Programs\Anac\lib\site-packages\numpy\__init__.py:133: RuntimeWarning: coroutine 'main' was never awaited
  from .version import git_revision as __git_revision__
Task exception was never retrieved
future: <Task finished name='Task-47' coro=<fun2() done, defined at C:\Users\vitya\AppData\Local\Temp/ipykernel_5256/897330587.py:7> exception=TypeError("object NoneType can't be used in 'await' expression")>
Traceback (most recent call last):
  File "C:\Users\vitya\AppData\Local\Temp/ipykernel_5256/897330587.py", line 9, in fun2
    await time.sleep(3)
TypeError: object NoneType can't be used in 'await' expression


In [38]:
def generateNodes(N):
    """ Generate random 3D nodes
    """
    
    return np.random.rand(N, 3)

def spCalcDistance(nodes):
    """ Single process calculation of the distance function.
    """
    
    p = nodes
    q = nodes.T
    
    # components of the distance vector        
    Rx = p[:, 0:1] - q[0:1]
    Ry = p[:, 1:2] - q[1:2]
    Rz = p[:, 2:3] - q[2:3]
    
    # calculate function of the distance
    L = np.sqrt(Rx * Rx + Ry * Ry + Rz * Rz)
    D = L * L * L / 12 + L * L / 6
    
    return D

In [ ]:
def mpCalcDistance_Worker(nodes, queue, arrD):
    """ Worker process for the multiprocessing calculations
    """

    nP = nodes.shape[0]
    nQ = nodes.shape[0]

    D = np.reshape(np.frombuffer(arrD), (nP, nQ))

    while True:
        job = queue.get()
        if job == None:
            break

        start = job[0]
        stop = job[0] + job[1]
          
        # components of the distance vector
        p = nodes[start:stop]
        q = nodes.T
        
        Rx = p[:, 0:1] - q[0:1]
        Ry = p[:, 1:2] - q[1:2]
        Rz = p[:, 2:3] - q[2:3]

        # calculate function of the distance
        L = np.sqrt(Rx * Rx + Ry * Ry + Rz * Rz)
        D[start:stop, :] = L * L * L / 12 + L * L / 6
        
        queue.task_done()
    queue.task_done()

def mpCalcDistance(nodes):
    """ Multiple processes calculation of the distance function.
    """

    # allocate shared array
    nP = nodes.shape[0]    
    nQ = nodes.shape[0]

    arrD = mp.RawArray(ctypes.c_double, nP * nQ)
   
    # setup jobs
    #nCPU = mp.cpu_count()
    nCPU = 2
    nJobs = nCPU * 36
   
    q = nP / nJobs
    r = nP % nJobs
 
    jobs = []
    firstRow = 0
    for i in range(nJobs):
        rowsInJob = q
        if (r > 0):
            rowsInJob += 1
            r -= 1
        jobs.append((firstRow, rowsInJob))
        firstRow += rowsInJob

    queue = mp.JoinableQueue()
    for job in jobs:
        queue.put(job)
    for i in range(nCPU):
        queue.put(None)

    # run workers
    workers = []
    for i in range(nCPU):
        worker = mp.Process(target = mpCalcDistance_Worker,
                            args = (nodes, queue, arrD))
        workers.append(worker)
        worker.start()

    queue.join()
   
    # make array from shared memory    
    D = np.reshape(np.frombuffer(arrD), (nP, nQ))
    return D

def compareTimes():
    """ Compare execution time single processing versus multiple processing.
    """
    nodes = generateNodes(3000)
    
    t0 = time.time()
    spD = spCalcDistance(nodes)
    t1 = time.time()
    print "single process time: {:.3f} s.".format(t1 - t0)

    t0 = time.time()
    mpD = mpCalcDistance(nodes)
    t1 = time.time()
    print "multiple processes time: {:.3f} s.".format(t1 - t0)
    
    err = np.linalg.norm(mpD - spD)
    print "calculate error: {:.2e}".format(err)
    
def showTimePlot():    
    """ Generate execution time plot single processing versus multiple processing.
    """
    
    N = range(100, 4000, 4)
    spTimes = []
    mpTimes = []
    rates = []
    for i in N:
        print i
        nodes = generateNodes(i)
        
        t0 = time.time()
        spD = spCalcDistance(nodes)
        t1 = time.time()
        sp_tt = t1 - t0
        spTimes.append(sp_tt)
        
        t0 = time.time()
        mpD = mpCalcDistance(nodes)
        t1 = time.time()
        mp_tt = t1 - t0
        mpTimes.append(mp_tt)
        
        rates.append(sp_tt / mp_tt)
                
    plt.figure()
    plt.plot(N, spTimes)
    plt.plot(N, mpTimes)
    plt.xlabel("N")
    plt.ylabel("Execution time")
    
    plt.figure()
    plt.plot(N, rates)
    plt.xlabel("N")
    plt.ylabel("Rate")
    plt.show()

def main():
    compareTimes()
    #showTimePlot()

if __name__ == '__main__':
    main()